# Activity Data

In [21]:
import pandas as pd
from pandas import json_normalize
import requests
import os

from dotenv import load_dotenv
load_dotenv()


from openapi_client import Configuration, ApiClient
from openapi_client.api.files_api import FilesApi
from openapi_client.api.spreadsheets_api import SpreadsheetsApi


import time  # Import time for managing token expiration


# Initialize global variables for token and expiration
bearer_token = None
token_expiration = 0

In [22]:
def get_access_token():
    global bearer_token, token_expiration
    
    # Check if the token is expired
    if bearer_token is None or time.time() >= token_expiration:
        host = 'api.eu.wdesk.com'
        path = '/iam/v1/oauth2/token'
        bearer_url = f'https://{host}{path}'
        bearer_headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'charset': 'UTF-8'
        }
        bearer_data = {
            'client_id': os.getenv("WORKIVA_CLIENT_ID"),
            'client_secret': os.getenv("WORKIVA_CLIENT_SECRET"),
            'grant_type': 'client_credentials'
        }

        # Requesting the access token
        bearer_result = requests.post(bearer_url, data=bearer_data, headers=bearer_headers)

        # Print status code and response content for debugging
        print("Token Request Status Code:", bearer_result.status_code)
        bearer_json = bearer_result.json()
        print("Token Request Response JSON:", bearer_json)

        # Check if access token is present
        bearer_token = bearer_json.get('access_token')
        
        # Set token expiration to 1 hour from now (3600 seconds)
        token_expiration = time.time() + 3600

    return bearer_token


# Create a configuration object
config = Configuration(
    host="https://api.eu.wdesk.com/platform/v1",
    refresh_token_callback=get_access_token
)

# Create an API client with the bearer token
config.access_token = get_access_token()


# Create an API client
api_client = ApiClient(configuration=config)

# Create an instance of the FilesApi
spreadsheets_api = SpreadsheetsApi(api_client)

# Example: Create a new file


# Call the get_files method
response = spreadsheets_api.get_spreadsheets()


# Display the response
print(response)


Token Request Status Code: 200
Token Request Response JSON: {'token_type': 'Bearer', 'expires_in': 600, 'access_token': 'eyJraWQiOiJUMEYxZEdneU1GUnZhMlZ1UzJWNVVHRnlaVzUwSGs5QmRYUm9NakJVYjJ0bGJrdGxlVWxFSGs5QmRYUm9NakJVYjJ0bGJrdGxlUjgxTmpJNU5EazVOVE0wTWpFek1USXciLCJhbGciOiJSUzUxMiJ9.eyJhdWQiOiIxOTAyNzI4OC01ZGIzLTRmNGEtODZjZS1kNjAyZGQ1MjBjNzIiLCJleHAiOjE3Mjg0NzU3MjksImdudCI6ImNsaWVudF9jcmVkZW50aWFscyIsImlhdCI6MTcyODQ3NTEyOSwiaXNzIjoiaHR0cHM6Ly9ldS53ZGVzay5jb20iLCJqdGkiOiJVTW5wZEIwbnJxSXdfVVMyM2V4ZnlnIiwic2NvcGUiOiJ0YXNrOndyaXRlIGRhdGFlbnRpdHl8ciBmaWxlOnJlYWQgZ3JhcGg6d3JpdGUgZmlsZTp3cml0ZSBncmFwaF9hcGl8dyBzY2ltfHIgc294fGFkbWluIGRhdGFfdGFibGVzfHcgYXVkaXRfYXBpfHIgYWN0aXZpdHk6cmVhZCBkYXRhX3RhYmxlc3xyIGF1ZGl0X2FwaXx3IGdyYXBoX2FwaXxyIGdyYXBoOnJlYWQgdGFzazpyZWFkIiwic2NwIjpbInRhc2s6d3JpdGUiLCJkYXRhZW50aXR5fHIiLCJmaWxlOnJlYWQiLCJncmFwaDp3cml0ZSIsImZpbGU6d3JpdGUiLCJncmFwaF9hcGl8dyIsInNjaW18ciIsInNveHxhZG1pbiIsImRhdGFfdGFibGVzfHciLCJhdWRpdF9hcGl8ciIsImFjdGl2aXR5OnJlYWQiLCJkYXRhX3RhYmxlc3xyIiwiYXVkaX

In [23]:
import pandas as pd

# Function to extract keys and values from a list of File objects
def extract_keys_values(data):
    return [extract_keys_values(item) for item in data] if isinstance(data, list) else data.__dict__

# Convert response data to a pandas DataFrame
extracted_data = extract_keys_values(response.data)
df = pd.json_normalize(extracted_data)

# Display the DataFrame
df

,id,name,template,created,modified,sheets
0,9aad509d8441489b9b95e6cc79cc0f61,Fund Code Fund Name Mapping File,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
1,5628febf9c064c9ea89cd3f56e2c3ab3,Spreadsheet_1000_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 9,...","user=None date_time=datetime.datetime(2024, 9,...",None
2,062c8fa241f6449988b7cccac3b8d0da,Spreadsheet_1415_31.03.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
3,d1c8c90cabd540c3a0272c84b44ae142,Spreadsheet_1415_31.03.2024 (1),False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
4,3043049e7a0d4f50a1a169345482313c,Spreadsheet_1800_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
5,13c18d7039fe4d3bbd6cc462635ee4e9,Spreadsheet_2100_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 9,...","user=None date_time=datetime.datetime(2024, 9,...",None
6,69aaaa3ac0a44da5ad40062f4c02db5d,Spreadsheet_2200_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
7,e0bd5db0332b4308b8c00ff70c95b042,Spreadsheet_2700_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
8,701a618099c5455880be7729f61572f8,Spreadsheet_2700_31.03.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
9,38cd6377499d4aeda4129cd5f21a93de,Spreadsheet_2700_31.07.2024,False,"user=None date_time=datetime.datetime(2024, 9,...","user=None date_time=datetime.datetime(2024, 9,...",None


In [24]:
# Filter the DataFrame to get only the rows where the 'name' column starts with 'Spreadsheet_'
filtered_spreadsheets_df = df[df['name'].str.startswith('Workiva_Spreadsheet_')]

# Display the filtered DataFrame
filtered_spreadsheets_df



,id,name,template,created,modified,sheets
28,505756aa0a9841578a7802f9043dbcf1,Workiva_Spreadsheet_1000_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 9,...","user=None date_time=datetime.datetime(2024, 9,...",None
29,806dbcadee3142f98136fe0c621efe07,Workiva_Spreadsheet_1415_31.03.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
30,1afee19b42cb4187b0e0737b1687e281,Workiva_Spreadsheet_1415_31.03.2024 (1),False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
31,7d6aad05cece403aafea4a8360b45b39,Workiva_Spreadsheet_1800_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
32,3e051b351b2d4fd399b3c2d65f2d9ef4,Workiva_Spreadsheet_2100_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 9,...","user=None date_time=datetime.datetime(2024, 9,...",None
33,d9c97cb4b28146f98f0a5b5b8cc4897e,Workiva_Spreadsheet_2200_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
34,607932bb40544904b49bcef7c67a980e,Workiva_Spreadsheet_2700_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 10...",None
35,6eae13131de9450cb6c41b17549cd92d,Workiva_Spreadsheet_2700_31.03.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None
36,7e3f092cc0c045aa9602954120a8e6b2,Workiva_Spreadsheet_2700_31.07.2024,False,"user=None date_time=datetime.datetime(2024, 9,...","user=None date_time=datetime.datetime(2024, 9,...",None
37,7234ebb4dae4461781091164fc010aee,Workiva_Spreadsheet_2800_30.06.2024,False,"user=None date_time=datetime.datetime(2024, 8,...","user=None date_time=datetime.datetime(2024, 8,...",None


In [32]:
# Initialize a list to hold DataFrames for each spreadsheet
dataframes = []

# Loop over the filtered spreadsheet IDs
for spreadsheet_id in filtered_spreadsheets_df['id']:
    # Retrieve the spreadsheet data
    spreadsheet_data = spreadsheets_api.get_sheets(spreadsheet_id)
    extracted_data = extract_keys_values(spreadsheet_data.data)
    spreadsheet_data = pd.json_normalize(extracted_data)  
    # print(spreadsheet_data.head())
    asset_sheet = spreadsheet_data[spreadsheet_data['name'] == "Portfolio Overview"]
    print(asset_sheet)

    # Define the path to save the file
    save_path = os.path.join(f'sheet_data_{spreadsheet_id}.txt')

    # Save the sheet details to the specified path
    with open(save_path, 'w') as file:
        file.write(asset_sheet.to_string())

    for sheet_id in asset_sheet['id']:
        sheet_details = spreadsheets_api.get_sheet_data(spreadsheet_id, sheet_id)
        # print(sheet_details)
        with open(f'sheet_data_{sheet_id}.txt', 'w') as file:
            file.write(sheet_details.to_str())
        # break
        # Convert the retrieved sheet data to a pandas DataFrame
        sheet_df = pd.json_normalize(sheet_details)
        # Append the DataFrame to the list
        dataframes.append(sheet_df)

    break
    

# # Optionally, concatenate all DataFrames into a single DataFrame
# all_spreadsheets_df = pd.concat(dataframes, ignore_index=True)

# # Display the combined DataFrame
# all_spreadsheets_df


                                  id                name  \
43  21ffc7681e784a7eaefa67b1fcb6e0e6  Portfolio Overview   

                                               parent  index  \
43  id='f81b8923cd35437b920584878812a18c' name='Da...      4   

                                             children dataset  
43  [id='62e2f51a68504331bf33cfe9f099617d' name='P...    None  
